<a href="https://colab.research.google.com/github/mathgurgel/TP_IA/blob/main/tp_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de Classificação

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

import pandas as pd

#### Logistic Regression

In [3]:
def logistic_regression():
    # Carrega os dados de treinamento e teste
    df_training = pd.read_csv('conjunto_de_dados/falencia-treinamento.csv')
    df_test = pd.read_csv('conjunto_de_dados/falencia-teste.csv')

    # Carrega o resultado esperado do modelo de teste
    df_test_result = pd.read_csv('conjunto_de_dados/modelo-submissao.csv')

    # Separa as variáveis independentes e dependentes (treinamento)
    independent_variables_training = df_training.drop('Resultado', axis='columns')
    dependent_variable_training = df_training['Resultado']

    # Recebe as variáveis independentes do conjunto de teste
    independent_variables_test = df_test

    # Cria o modelo de regressão logística
    logistic_regression = LogisticRegression()

    # Treina o modelo
    logistic_regression.fit(independent_variables_training, dependent_variable_training)

    # Faz a previsão do modelo
    prediction = logistic_regression.predict(independent_variables_test) # Resultado do modelo

    # Calcula a acurácia do modelo
    accuracy = accuracy_score(df_test_result['Resultado'], prediction)
    f_measure = f1_score(df_test_result['Resultado'], prediction)

    print('Acurácia: ', accuracy)
    print('F-Measure: ', f_measure)

#### Decision Tree

#### Random Forest

#### Resultados

In [4]:
logistic_regression()

Acurácia:  0.73
F-Measure:  0.4489795918367347
